## Programming Assignment - Bootstrap Methods - Adv. Econometrics 2

**Deadline**:  Friday 17:00 hours, 8 January 2021

|Nr|**Name**|**Student ID**|**Email**|
|--|--------|--------------|---------|
|1.|Dante van der Heijden|11020075|         |
|2.|Wietse Steenstra|1100|         |
|3.|Willem Kullberg|              |         |

**Declaration of Originality**

We whose names are given under 1., 2. and 3. above declare that:

1. These solutions are solely our own work.
2. We have not made (part of) these solutions available to any other student.
3. We shall not engage in any other activities that will dishonestly improve my results or dishonestly improve or hurt the results of others.

## Instructions for completing and submitting the assignment
1. Submit your work in the form of (i) a Jupyter Notebook and (ii) PDF-file via Canvas assuming basic econometric knowledge, before the deadline. Your notebook should not give errors when executed with `Run All`.
2. Complete the table with the info of your group members. By submitting the Jupyter Notebook, you agree with the included declaration of originality. Do not copy work of others. This will be considered as fraud!
3. Clarify your code with comments.

## Hints
- Only use the paired bootstrap
- Work with Numpy vectors or matrices as much as possible, e.g. `np.quantile(tB_OLS,[0.05,0.95],axis=0)` returns two quantiles for the whole vector of OLS estimates
- When coding, you can reduce the running time by setting `BOOTREP=99` and reduce the number of simulations. For the final execution, please return to original values!
- For a progress bar, please install `conda install -c conda-forge tqdm` or if you don't use anaconda use can just execute `pip install tqdm`
- If you want to use plotly, please install `conda install -c plotly plotly`
- Below, you can find Python code for generating the data and doing a simulation using multicores. To use multicores, you have to install `multiprocess`: `conda install -c conda-forge multiprocess`. Otherwise, execute `pip install multiprocess`
- The idea behind multiprocess is that each CPU core receives a sample, executes the resampling and returns the results. These results will be stored in one big list, which can be analyzed after the simulation.

## Assignment 

The purpose of this assignment is for you to gain practical experience with resampling methods. You will investigate several bootstrap confidence intervals for OLS and LASSO estimators. The DGP is given by:

- $X_i \sim N(0,\Sigma)$, $\Sigma=(\sigma_{ij}) \in \mathbb{R}^{p\times p}$ with $\sigma_{ij}=\rho^{|i-j|}$, $\beta_j=0$ for $1\leq j\leq p-15$, $\beta_j=0.5$ for $p-14\leq j\leq p-10$, $\beta_j=1.5$ for $p-9\leq j \leq p-5$ $\beta_j=2.5$ for $p-4 \leq j \leq p$. 

- $\varepsilon_1,...,\varepsilon_n \sim N(0,1)$

- $y=X \beta+\varepsilon$

Let $\hat{\beta}=(X'X)^{-1}X'y$ denote the OLS estimator, while $\breve{\beta}$ denote the LASSO estimator based on minimizing
$$ \sum_{i=1}^{n} (y_i- b'X_i)^2+\alpha \sum_{j=1}^{p}|b_j|.$$
Only consider `lasso = linear_model.Lasso(alpha=0.02)`, so keep the amount of regularization fixed!

Please, briefly answer all questions below using graphs and if necessary tables.

## Import packages

In [1]:
import numpy as np
from sklearn import linear_model
#import plotly.express as px        # uncomment if you want to use plotly.express
from tqdm.notebook import tqdm
import pandas as pd

## Generate Samples

In [2]:
REP = 1000                            # numer of Monte Carlo simulations
BOOTREP = 99                         # number of bootstrap replications SET BACK TO 999
n = 50
p = 25
rho = 0.6
mu = np.zeros(p)
Sigma = np.identity(p)
for i in range(p):
    for j in range(p):
        Sigma[i,j] = rho**abs(i - j)
beta = np.zeros(p)
beta[(p - 15):(p - 10)] = 0.5
beta[(p - 10):(p - 5)] = 1.5
beta[(p - 5):] = 2.5
arglist=[]
for r in tqdm(range(REP)):
    X = np.random.multivariate_normal(mean=mu, cov=Sigma, size=n)
    eps = np.random.normal(size=n)
    y = X@beta + eps
    arglist.append((r,BOOTREP,y,X))

  0%|          | 0/1000 [00:00<?, ?it/s]

## Resampling Procedure

In [3]:
def Bootstrap(args):
    (iter,BOOTREP,y,X)=args
    
    # Define OLS function
    def OLS(y,X):
        N,p = X.shape                   # number of observations and regressors
        XXi = np.linalg.inv(X.T @ X)
        b_ols = XXi @ (X.T @ y)
        res = y-X @ b_ols
        s2 = (res @ res)/(N-p)
        SE = np.sqrt(s2*np.diag(XXi))
        return b_ols,SE,res
    
    import numpy as np
    from sklearn import linear_model
    from scipy.stats import norm
    
    n,p = X.shape
    # Estimates original sample
    lasso = linear_model.Lasso(alpha=0.02)
    lasso.fit(X, y)
    b_LASSO=np.copy(lasso.coef_)
    b_OLS,b_OLS_SE,res = OLS(y,X)
    # initilize bootstrap arrays
    bB_LASSO = np.zeros((BOOTREP,p))
    bB_OLS   = np.zeros((BOOTREP,p))
    #
    #
    np.random.seed(1)
    # balanced bootstap
    index_B=np.random.permutation(np.repeat(np.arange(n),BOOTREP)).reshape((BOOTREP,n))
    for b in range(BOOTREP):
        index = index_B[b,:]  # select the indices
        yB = np.copy(y[index])
        XB = np.copy(X[index,:])
        lasso.fit(XB, yB)
        bB_LASSO[b,:] = np.copy(lasso.coef_)
        # other code
        bB_OLS[b,:] = OLS(yB, XB)[0]
        
    se_boot_OLS = np.std(bB_OLS, axis=0)
    se_boot_LASSO = np.std(bB_LASSO, axis=0)
    
    # percentile
    q_bB_LASSO = np.quantile(bB_LASSO,[0.05,0.95],axis=0)
    q_bB_OLS   = np.quantile(bB_OLS,[0.05,0.95],axis=0)
    argout = [b_LASSO,b_OLS, se_boot_LASSO, se_boot_OLS     # add more when necessary
             ]
    return(argout)

## Execute the Simulation and get Results

In [4]:
from multiprocess import Pool
pool4 = Pool(processes=4)
result_list = list(tqdm(pool4.imap_unordered(Bootstrap, arglist), total=REP))
pool4.close()
pool4.join()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
result_list[1][2]

array([0.15288023, 0.22340839, 0.19624792, 0.23125549, 0.18301229,
       0.19203269, 0.19846431, 0.26484967, 0.19006762, 0.21435803,
       0.27364855, 0.27310009, 0.28346724, 0.29917037, 0.21371885,
       0.33993381, 0.33704587, 0.26236735, 0.31774392, 0.41138643,
       0.34125059, 0.32869807, 0.37300222, 0.32872967, 0.25181983])

## Perform the Post-Processing

In [11]:
nr_methods=4                        # number of methods
                                    # 0 = percentile
                                    # 1 = SE_boot
                                    # 2 = percentile-t
                                    # 3 = BCa
b_LASSO    =np.zeros((REP,p))
b_OLS      =np.zeros((REP,p))
se_boot_LASSO    =np.zeros((REP,p))
se_boot_OLS      =np.zeros((REP,p))
for r in tqdm(range(REP)):
    b_LASSO[r,:]  = result_list[r][0]
    b_OLS[r,:]    = result_list[r][1]
    se_boot_LASSO[r] = result_list[r][2]
    se_boot_OLS[r] = result_list[r][3]

  0%|          | 0/1000 [00:00<?, ?it/s]

1. Choose $n=50$, $p=25$ and $\rho=0.6$. Determine the bias and RMSE of the OLS and LASSO estimators using 1,000 Monte Carlo replications.

## Carry Out Analysis

In [12]:
##### 1.

MC_b_OLS, MC_b_LASSO = b_OLS, b_LASSO

# Calculate bias for each regressor
MC_mean_OLS = np.mean(MC_b_OLS, axis = 0)
MC_mean_LASSO = np.mean(MC_b_LASSO, axis = 0)
MC_bias_OLS = MC_mean_OLS - beta
MC_bias_LASSO = MC_mean_LASSO - beta

# Calculate SE for each regressor
MC_SE_OLS = np.std(MC_b_OLS, axis = 0)
MC_SE_LASSO = np.std(MC_b_LASSO, axis = 0)

# Mean absolute bias
avg_bias_OLS = np.mean(abs(MC_bias_OLS))
avg_bias_LASSO = np.mean(abs(MC_bias_LASSO))

# Mean SE
avg_SE_OLS = np.mean(MC_SE_OLS)
avg_SE_LASSO = np.mean(MC_SE_LASSO)

print("Avg OLS bias = {0}, Avg Lasso bias = {1}".format(avg_bias_OLS,avg_bias_LASSO))
print("Avg OLS SE = {0}, Avg Lasso SE = {1}".format(avg_SE_OLS,avg_SE_LASSO))

Avg OLS bias = 0.005959321848934145, Avg Lasso bias = 0.009728625382472121
Avg OLS SE = 0.2954969432736685, Avg Lasso SE = 0.24387646524823148


2. Estimate by simulation the coverage probabilities (cov. prob.) of the 90% first-order asymptotic two-sided confidence intervals for the OLS estimator, i.e. the fraction of confidence intervals (CI)
$$[\hat{\beta_j}-1.645 SE(\hat{\beta_j}),\hat{\beta_j}+1.645 SE(\hat{\beta_j})]$$
that contains the true parameter $\beta_j$. Here $SE(\hat{\beta_j})$ is the usual (non-robust) standard error based on $s^2(X'X)^{-1}$.

In [13]:
##### 2. door Dante 
mc_ols_lb = pd.DataFrame(MC_b_OLS) - 1.645*MC_SE_OLS
mc_ols_ub = pd.DataFrame(MC_b_OLS) + 1.645*MC_SE_OLS

mc_ols_CI = mc_ols_lb.le(pd.Series(beta).T, axis=1) & mc_ols_ub.ge(pd.Series(beta).T, axis=1)
mc_ols_cov_avg_prob = np.mean(mc_ols_CI.sum()/REP)

print("Average coverage probability = {}".format(mc_ols_cov_avg_prob))

Average coverage probability = 0.90256


In [14]:
##### 2. door Wietse

# Get upper and lower bound of confidence interval
CI_lower_OLS = MC_mean_OLS - 1.645 * MC_SE_OLS
CI_upper_OLS = MC_mean_OLS + 1.645 * MC_SE_OLS

# Create dataframe to check whether b value falls within confidence interval
df_larger_than_lower_bound_OLS = pd.DataFrame(MC_b_OLS).ge(pd.Series(CI_lower_OLS), axis = 1)
df_smaller_than_upper_bound_OLS = pd.DataFrame(MC_b_OLS).le(pd.Series(CI_upper_OLS), axis = 1)
df_CI_OLS = df_larger_than_lower_bound_OLS & df_smaller_than_upper_bound_OLS

# Count number of times b falls within confidence interval per regressor
cov_prob_OLS = df_CI_OLS.sum()/REP

# Get average coverage probability
avg_cov_prob_OLS = np.mean(cov_prob_OLS)

print("Average coverage probability = {}".format(avg_cov_prob_OLS))

Average coverage probability = 0.9028000000000002


3. Estimate by simulation the cov. prob. of the 90% first-order asymptotic two-sided CI for OLS and LASSO using $$SE_{boot}(\tilde{\beta}),$$
for $$\tilde{\beta}\in \{\hat{\beta},\breve{\beta}\}.$$

In [16]:
##### 3. door Dante
SE_Boot_OLS = se_boot_OLS
SE_Boot_LASSO = se_boot_LASSO

boot_ols_lb = pd.DataFrame(b_OLS) - 1.645*SE_Boot_OLS[0]
boot_ols_ub = pd.DataFrame(b_OLS) + 1.645*SE_Boot_OLS[0]
boot_lasso_lb = pd.DataFrame(b_LASSO) - 1.645*SE_Boot_LASSO[0]
boot_lasso_ub = pd.DataFrame(b_LASSO) + 1.645*SE_Boot_LASSO[0]

boot_ols_CI = boot_ols_lb.le(pd.Series(beta).T, axis=1) & boot_ols_ub.ge(pd.Series(beta).T, axis=1)
boot_ols_cov_avg_prob = np.mean(boot_ols_CI.sum()/REP)
boot_lasso_CI = boot_lasso_lb.le(pd.Series(beta).T, axis=1) & boot_lasso_ub.ge(pd.Series(beta).T, axis=1)
boot_lasso_cov_avg_prob = np.mean(boot_lasso_CI.sum()/REP)

print("Average coverage probability OLS = {}".format(boot_ols_cov_avg_prob))
print("Average coverage probability LASSO = {}".format(boot_lasso_cov_avg_prob))

Average coverage probability OLS = 0.9964799999999999
Average coverage probability LASSO = 0.9489199999999998


In [ ]:
##### 3. door Wietse

# Combine regressor estimates of OLS and Lasso
# @Dante/Willem is dit wat ze bedoelen met beta_tilde denken jullie?
MC_b_combined = np.concatenate((MC_b_OLS, MC_b_LASSO), axis = 1)

# Calculate mean and SE of combined estimator
MC_mean_combined = np.mean(MC_b_combined, axis = 0)
MC_SE_combined = np.std(MC_b_combined, axis = 0)

# Get upper and lower bound of confidence interval
CI_lower_combined = MC_mean_combined - 1.645 * MC_SE_combined
CI_upper_combined = MC_mean_combined + 1.645 * MC_SE_combined

# Create dataframe to check whether b value falls within confidence interval
df_larger_than_lower_bound_combined = pd.DataFrame(MC_b_combined).ge(pd.Series(CI_lower_combined), axis = 1)
df_smaller_than_upper_bound_combined = pd.DataFrame(MC_b_combined).le(pd.Series(CI_upper_combined), axis = 1)
df_CI_combined = df_larger_than_lower_bound_combined & df_smaller_than_upper_bound_combined

# Count number of times b falls within confidence interval per regressor
cov_prob_combined = df_CI_combined.sum()/REP

# Get average coverage probability
avg_cov_prob_combined = np.mean(cov_prob_combined)

print("Average coverage probability = {}".format(avg_cov_prob_combined))

4. Note that the cov. prob. can be interpreted as the number of successes in 1,000 trials. Hence, the estimated cov. prob. is not significantly different (at the 95% confidence level) from 90% if its value is contained in the interval
$$0.90\pm 1.96\sqrt{(0.90\times 0.10/1000)}=[0.8814,0.9186].$$
Check if the estimated cov. prob. of questions 2 & 3 are significantly different from 90%. What is your conclusion?

In [17]:
print("It is clear that the bootstrapped convergence probability is way higher than that of the non-bootstrapped.")

It is clear that the bootstrapped convergence probability is way higher than that of the non-bootstrapped.


In [ ]:
##### 4. Wietse

# @Dante/Willem Weet niet of dit klopt, je zou zeggen dat ze nu nog verschillen van 90% zodat je dat kunt oplossen met bootstrap?
print("Zowel in 2 en 3 vallen de coverage probabilities binnen dit interval, dus ze verschillen niet significant van 90%")

5. Estimate by simulation the cov. prob. for OLS and LASSO of 90% equal-tailed two-sided percentile bootstrap confidence intervals:
$$(\tilde{\beta}_{95\%}^*,\tilde{\beta}_{5\%}^*),$$
where $\mathbb{P}_*[\tilde{\beta}^*>\beta_{5\%}^*]=5\%$ and  $\tilde{\beta}\in \{\hat{\beta},\breve{\beta}\}$.

6.  Estimate by simulation the cov. prob. for OLS of 90% equal-tailed two-sided percentile-$t$ bootstrap confidence intervals based on the quantiles of the root
$$(\hat{\beta}^*-\hat{\beta})/SE(\hat{\beta}^*),$$ 
where $SE(\hat{\beta}^*)$ is based on $s^{*2}(X^{*}\,' X^{*})^{-1}$.

7. What problems would you encounter if you wanted to implement the percentile-$t$ intervals for the LASSO? How could you remedy these problems (you don't have to implement this)?

8.  Estimate by simulation the cov. prob. for LASSO of 90% two-sided bias-corrected and accelerated (BC$_a$) confidence intervals. For this, use the bootstrap to estimate the (median) bias and the Jackknife for the acceleration constant. In the BC$_a$ method, the quantiles are adjusted:
$$\alpha_1=\Phi\left ( \hat{z}_0+\frac{\hat{z}_0+z_{\alpha/2}}{1-\hat{a}(\hat{z}_0+z_{\alpha/2})} \right ), \\
  \alpha_2=\Phi\left ( \hat{z}_0+\frac{\hat{z}_0+z_{1-\alpha}}{1-\hat{a}(\hat{z}_0+z_{1-\alpha/2})} \right ) \\
$$
with $z_{0.95}=1.645$. Here
$$\hat{z}_0=\Phi^{-1}\left ( \frac{\sum_{i=1}^n \mathbb{1} \{\hat{\theta}^*(b)<\hat{\theta}\}}{B} \right)$$
and
$$ \hat{a}=\frac{\sum_{i=1}^n (\hat{\theta}_{(\cdot)}-\hat{\theta}_{(i)})^3}{6\{\sum_{i=1}^n(\hat{\theta}_{(\cdot)}-\hat{\theta}_{(i)})^2\}^{3/2}}$$
with $\hat{\theta}_{(\cdot)}=\sum_{i=1}^n \hat{\theta}_{(i)}/n$; see for more details Section 14.3 of Efron, B., & Tibshirani, R. J. (1994). An introduction to the bootstrap: [link](http://www.ru.ac.bd/stat/wp-content/uploads/sites/25/2019/03/501_02_Efron_Introduction-to-the-Bootstrap.pdf)).

9. Based on all the results of this assignment, which inference procedure would you advise a practitioner that wants to conduct inference in a model described by the DGP? Motivate your recommendation.